<a href="https://colab.research.google.com/github/sambitmishra98/PyFR-ideal-performance/blob/main/performance_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projected performance computation from mesh

In this document, we aim to compare the performance of expected PyFR performance in comparison with actual performance.

The expected performance is computed by understanding inputs and outputs to all kernels used for computation.

The actual performance is computed using `perf_counter()` and looking at wall-time in the solution file. For example, actual performance of Intel MAX GPUs is shown in a [Google Docs file](https://docs.google.com/document/d/1yX7JqTTsXRikTtzRon-03TgRGceByce075N4-Ptp7cI/edit?usp=sharing) (Restricted access). The latter method was used to benchmark performance of PyFR on A100 GPUs in a paper: [Scaling Study of Flow Simulations on Composable Cyberinfrastructure](https://doi.org/10.1145/3569951.3597565).

## Performance details from configuration file

Following the configurations as given in [PyFR documentation](https://pyfr.readthedocs.io/en/latest/user_guide.html#configuration-file-ini).
Only those relevant to performance computation is declared below.


In [1]:
# [backend]
precision = 'double'

# [solver]
system = 'navier-stokes'
order = 1

# [solver-time-integrator]
scheme = 'rk4'
tstart = 0
tend = 1.0001
dt = 0.0001

## Processing data from mesh


Details of how mesh size may be obtained is found in plugin path `pyfr/plugins/benchmark.py` in benchmark branch in [sambitmihsra98/PyFR.git](https://github.com/sambitmishra98/PyFR.git).


In [2]:
element_counts = { 'tet': 1,
}

## element-wise degrees of freedom calculation

An analysis of Flux Reconstruction schemes on Tetrahedral elements may be found in [this paper](https://doi.org/10.1007/s10915-016-0204-y). In gist, we have solution points and flux points for tetrahedra as

Tetrahedra:
$$N^n_s = \frac{(n+1)(n+2)(n+3)}{6} $$
$$N^n_f = \frac{4\cdot(n+1)(n+2)(n+3)}{6} $$

for polynomial of order $n$.

In [ ]:
def element_dof(etype, n):
    if etype == 'tet':
        Nu = (n+1)*(n+2)*(n+3)/6
        Nf = 4*Nu
    else:
        raise Exception("Not implemented yet")

    return Nu, Nf
